<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [28]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

stock_ids = stock_n100

mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE',
       'BANKBARODA', 'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA',
       'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB',
       'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'IBREALEST',
       'ICICIBANK', 'ICICIPRULI', 'INDIANB', 'INDIGO', 'INDUSINDBK',
       'INDUSTOWER', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD', 'KOTAKBANK',
       'LT', 'LTIM', 'M&M', 'MARUTI', 'MCDOWELL-N', 'MOTILALOFS',
       'MUTHOOTFIN', 'NAM-INDIA', 'NATIONALUM', 'NAUKRI', 'PFIZER',
       'PIDILITIND', 'PVR', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'VOLTAS', 'WHIRLPOOL',
       'WIPRO', 'ZEEL'], dtype=object)

In [29]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    60
1    13
Name: InPortfolio, dtype: int64

In [30]:
# consistent compounders
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0    8
1    5
Name: CCIFlag, dtype: int64

In [31]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [32]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [33]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Upside%52W'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Stage'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Stage'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [34]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Upside%52W
Date,,,,,,,,,,,
2023-01-02 00:00:00+05:30,18197.0,^NSEI,4.92,0.77,0.40,18812.0,15294.0,82.53,17.47,48.0,3.0
2023-01-09 00:00:00+05:30,18101.0,^NSEI,4.08,0.66,0.19,18812.0,15294.0,79.80,20.20,47.0,4.0
2023-01-16 00:00:00+05:30,17895.0,^NSEI,2.60,0.56,-0.10,18812.0,15294.0,73.93,26.07,42.0,5.0
2023-01-23 00:00:00+05:30,18119.0,^NSEI,3.38,0.76,-0.07,18812.0,15294.0,80.29,19.71,51.0,4.0
2023-01-31 00:00:00+05:30,17662.0,^NSEI,0.35,0.90,-0.30,18812.0,15294.0,67.32,32.68,38.0,7.0


In [35]:
# Upside% Calculations
df_prec_dev['Upside%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev['Upside%Avg'] = round((df_prec_dev['Upside%FC'] + df_prec_dev['Upside%52W'])/2,0)
#df_prec_dev['Upside%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Curr_PE']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%FC', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [36]:
# portfolio stocks
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Upside%FC', 'Upside%52W', 'Upside%Avg', 'EstmtFlag','DuraFlag','Stage']
df_prec_dev = df_prec_dev[cols]
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'EstmtFlag','DuraFlag','Stage']

mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Profit%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%52W'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%Avg'] = round((tmp_df['Profit%FC'] + tmp_df['Profit%52W'])/2,0)
print('qualified stocks: '+str(tmp_df.shape[0]))

cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Profit%FC', 'Profit%52W', 'Profit%Avg', 'EstmtFlag','DuraFlag','Stage']
tmp_df = tmp_df[cols]
tmp_df.sort_values(by = 'Profit%Avg', ascending=False)

qualified stocks: 13


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
23,VOLTAS.NS,803.0,-10.42,-1.45,-0.61,1315.0,745.0,10.11,89.89,52.0,976.0,821.00,43.4,151.00,19.0,60.0,40.0,0.0,1.0,D
9,JUBLFOOD.NS,487.0,-13.83,-0.18,-1.35,687.0,463.0,10.65,89.35,36.0,636.0,499.45,87.9,67.90,27.0,38.0,32.0,1.0,1.0,B
37,TATAMOTORS.NS,452.0,5.88,0.23,0.24,518.0,372.0,54.86,45.14,70.0,509.0,392.00,19.2,1000.00,30.0,32.0,31.0,1.0,-1.0,B
3,MOTILALOFS.NS,681.0,-6.40,-0.27,0.14,937.0,654.0,9.68,90.32,40.0,915.0,733.00,19.7,9.47,25.0,28.0,26.0,-1.0,0.0,D
45,TECHM.NS,1015.0,-1.46,0.49,-0.19,1520.0,935.0,13.68,86.32,44.0,1124.0,1048.00,18.7,19.00,7.0,45.0,26.0,-1.0,1.0,B
29,MUTHOOTFIN.NS,1044.0,-1.41,0.19,-0.46,1447.0,956.0,17.94,82.06,45.0,1227.0,1082.00,12.5,11.10,13.0,34.0,24.0,-1.0,0.0,B
10,ICICIPRULI.NS,452.0,-11.47,-0.39,-0.22,595.0,440.0,8.00,92.00,39.0,594.0,484.00,60.9,85.00,23.0,23.0,23.0,1.0,0.0,D
8,BATAINDIA.NS,1529.0,-13.11,-0.34,-0.98,2011.0,1496.0,6.48,93.52,34.0,1998.0,1632.00,62.3,63.40,22.0,23.0,22.0,1.0,1.0,D
31,BERGEPAINT.NS,549.0,-9.98,-0.16,-1.02,739.0,540.0,4.60,95.40,38.0,637.0,570.00,72.9,56.40,12.0,30.0,21.0,1.0,1.0,B
15,NAUKRI.NS,3671.0,-8.02,0.10,-0.77,5096.0,3363.0,17.77,82.23,41.0,4684.0,4154.00,78.8,15.90,13.0,23.0,18.0,1.0,0.0,B


In [37]:
# wait for max profit

tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
9,JUBLFOOD.NS,487.0,-13.83,-0.18,-1.35,687.0,463.0,10.65,89.35,36.0,636.0,499.45,87.9,67.9,27.0,38.0,32.0,1.0,1.0,B
8,BATAINDIA.NS,1529.0,-13.11,-0.34,-0.98,2011.0,1496.0,6.48,93.52,34.0,1998.0,1632.00,62.3,63.4,22.0,23.0,22.0,1.0,1.0,D
31,BERGEPAINT.NS,549.0,-9.98,-0.16,-1.02,739.0,540.0,4.60,95.40,38.0,637.0,570.00,72.9,56.4,12.0,30.0,21.0,1.0,1.0,B
22,HDFCLIFE.NS,579.0,3.55,0.36,0.85,642.0,502.0,54.89,45.11,42.0,709.0,578.00,89.3,82.4,23.0,11.0,17.0,1.0,1.0,A
35,INFY.NS,1534.0,1.91,0.60,-0.32,1866.0,1351.0,35.49,64.51,52.0,1742.0,1628.00,23.0,27.0,7.0,15.0,11.0,1.0,1.0,A


In [38]:
# keep a watch
tmp_df[((tmp_df['EstmtFlag'] == 0) & (tmp_df['DuraFlag'] != -1)) | ((tmp_df['DuraFlag'] == 0) & (tmp_df['EstmtFlag'] != -1))].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
23,VOLTAS.NS,803.0,-10.42,-1.45,-0.61,1315.0,745.0,10.11,89.89,52.0,976.0,821.0,43.4,151.0,19.0,60.0,40.0,0.0,1.0,D
10,ICICIPRULI.NS,452.0,-11.47,-0.39,-0.22,595.0,440.0,8.00,92.00,39.0,594.0,484.0,60.9,85.0,23.0,23.0,23.0,1.0,0.0,D
15,NAUKRI.NS,3671.0,-8.02,0.10,-0.77,5096.0,3363.0,17.77,82.23,41.0,4684.0,4154.0,78.8,15.9,13.0,23.0,18.0,1.0,0.0,B


In [39]:
# sell as soon as you can
tmp_df[(tmp_df['EstmtFlag'] == -1) | (tmp_df['DuraFlag'] == -1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
37,TATAMOTORS.NS,452.0,5.88,0.23,0.24,518.0,372.0,54.86,45.14,70.0,509.0,392.0,19.2,1000.00,30.0,32.0,31.0,1.0,-1.0,B
3,MOTILALOFS.NS,681.0,-6.40,-0.27,0.14,937.0,654.0,9.68,90.32,40.0,915.0,733.0,19.7,9.47,25.0,28.0,26.0,-1.0,0.0,D
45,TECHM.NS,1015.0,-1.46,0.49,-0.19,1520.0,935.0,13.68,86.32,44.0,1124.0,1048.0,18.7,19.00,7.0,45.0,26.0,-1.0,1.0,B
29,MUTHOOTFIN.NS,1044.0,-1.41,0.19,-0.46,1447.0,956.0,17.94,82.06,45.0,1227.0,1082.0,12.5,11.10,13.0,34.0,24.0,-1.0,0.0,B
53,DIVISLAB.NS,3315.0,-6.58,-0.34,0.18,4530.0,3266.0,3.90,96.10,43.0,3558.0,3671.0,45.2,29.40,-3.0,23.0,10.0,-1.0,1.0,B


In [40]:
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Med_PE', 'Curr_PE','Upside%FC', 'Upside%52W', 'Upside%Avg','EstmtFlag','DuraFlag','Stage']
df_prec_dev = df_prec_dev[cols]

In [41]:
# prospect consistent compounders (CC)
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 5


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
9,JUBLFOOD.NS,487.0,-13.83,-0.18,-1.35,687.0,463.0,10.65,89.35,36.0,636.0,87.9,67.9,31.0,41.0,36.0,1.0,1.0,B
31,BERGEPAINT.NS,549.0,-9.98,-0.16,-1.02,739.0,540.0,4.60,95.40,38.0,637.0,72.9,56.4,16.0,35.0,26.0,1.0,1.0,B
54,LTIM.NS,4369.0,-2.88,0.47,-1.13,6333.0,3761.0,23.62,76.38,52.0,4663.0,26.6,37.4,7.0,45.0,26.0,1.0,0.0,O
30,ASIANPAINT.NS,2726.0,-13.49,0.32,-1.06,3566.0,2576.0,15.14,84.86,27.0,3180.0,73.4,68.4,17.0,31.0,24.0,1.0,1.0,T
18,TITAN.NS,2377.0,-4.80,0.88,-0.99,2770.0,1929.0,53.29,46.71,39.0,2980.0,78.0,66.6,25.0,17.0,21.0,1.0,1.0,O


In [42]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 11


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
5,WHIRLPOOL.NS,1391.0,-13.60,-0.25,-0.74,1845.0,1391.0,0.02,99.98,31.0,1854.0,72.2,68.0,33.0,33.0,33.0,1.0,0.0,O
17,BAJAJFINSV.NS,1342.0,-13.28,0.78,-2.24,1805.0,1093.0,35.04,64.96,30.0,1691.0,39.2,35.7,26.0,35.0,30.0,1.0,0.0,D
16,BAJFINANCE.NS,5886.0,-12.65,0.47,-1.55,7720.0,5265.0,25.31,74.69,39.0,7391.0,50.8,33.1,26.0,31.0,28.0,1.0,1.0,T
54,LTIM.NS,4369.0,-2.88,0.47,-1.13,6333.0,3761.0,23.62,76.38,52.0,4663.0,26.6,37.4,7.0,45.0,26.0,1.0,0.0,O
30,ASIANPAINT.NS,2726.0,-13.49,0.32,-1.06,3566.0,2576.0,15.14,84.86,27.0,3180.0,73.4,68.4,17.0,31.0,24.0,1.0,1.0,T
11,ICICIBANK.NS,832.0,-3.94,1.27,-0.78,953.0,650.0,60.03,39.97,34.0,1094.0,24.2,18.2,31.0,15.0,23.0,1.0,1.0,O
14,INDUSINDBK.NS,1083.0,-2.70,2.16,-0.08,1264.0,771.0,63.28,36.72,28.0,1402.0,22.8,12.4,29.0,17.0,23.0,1.0,1.0,O
33,DMART.NS,3502.0,-14.39,0.00,-1.13,4577.0,3231.0,20.16,79.84,33.0,4006.0,138.7,96.9,14.0,31.0,22.0,1.0,1.0,T
48,TATAPOWER.NS,213.0,-4.10,0.00,-0.94,287.0,195.0,19.46,80.54,58.0,232.0,20.4,26.7,9.0,35.0,22.0,1.0,0.0,O
12,SBIN.NS,554.0,-0.81,1.61,-0.67,626.0,434.0,62.24,37.76,35.0,716.0,13.2,12.0,29.0,13.0,21.0,1.0,0.0,O


In [43]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 79]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 4


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
5,WHIRLPOOL.NS,1391.0,-13.60,-0.25,-0.74,1845.0,1391.0,0.02,99.98,31.0,1854.0,72.2,68.0,33.0,33.0,33.0,1.0,0.0,O
30,ASIANPAINT.NS,2726.0,-13.49,0.32,-1.06,3566.0,2576.0,15.14,84.86,27.0,3180.0,73.4,68.4,17.0,31.0,24.0,1.0,1.0,T
48,TATAPOWER.NS,213.0,-4.10,0.00,-0.94,287.0,195.0,19.46,80.54,58.0,232.0,20.4,26.7,9.0,35.0,22.0,1.0,0.0,O
33,DMART.NS,3502.0,-14.39,0.00,-1.13,4577.0,3231.0,20.16,79.84,33.0,4006.0,138.7,96.9,14.0,31.0,22.0,1.0,1.0,T


In [44]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 7


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
56,M&M.NS,1379.0,9.95,1.75,0.55,1379.0,684.0,99.96,0.04,71.0,1475.0,20.5,20.50,7.0,0.0,4.0,1.0,1.0,O
69,INDIANB.NS,305.0,33.66,4.39,0.70,305.0,127.0,99.86,0.14,62.0,284.0,7.4,7.49,-7.0,0.0,-4.0,1.0,1.0,O
66,ITC.NS,352.0,7.75,1.53,0.00,361.0,204.0,94.49,5.51,69.0,350.0,NaN,25.40,-1.0,3.0,1.0,1.0,0.0,O
62,SUNPHARMA.NS,1034.0,8.55,1.57,0.29,1052.0,791.0,93.30,6.70,54.0,1056.0,NaN,30.90,2.0,2.0,2.0,1.0,1.0,O
65,HCLTECH.NS,1122.0,13.12,1.21,0.28,1148.0,857.0,91.13,8.87,63.0,1117.0,16.3,21.00,-0.0,2.0,1.0,1.0,1.0,A
39,FEDERALBNK.NS,134.0,10.20,2.46,0.00,140.0,82.0,90.43,9.57,49.0,152.0,11.0,10.20,13.0,4.0,8.0,1.0,0.0,A
71,BRITANNIA.NS,4318.0,9.22,1.77,0.57,4526.0,2995.0,86.39,13.61,47.0,3935.0,NaN,66.00,-9.0,5.0,-2.0,1.0,0.0,A


In [45]:
# prospects turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > -.5) & (tmp_df['Slope%_150'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
5,WHIRLPOOL.NS,1391.0,-13.6,-0.25,-0.74,1845.0,1391.0,0.02,99.98,31.0,1854.0,72.2,68.0,33.0,33.0,33.0,1.0,0.0,O
24,RELIANCE.NS,2354.0,-6.7,-0.40,-0.86,2811.0,2229.0,21.45,78.55,30.0,2854.0,25.3,25.0,21.0,19.0,20.0,1.0,0.0,O


In [46]:
# prospects turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage


In [47]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage


In [48]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage


In [49]:
# star stocks performance

star_stocks = ['BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS', 'CANBK.NS',
       'AUBANK.NS', 'TCS.NS', 'INDUSINDBK.NS', 'ICICIBANK.NS', 'WIPRO.NS',
       'AXISBANK.NS', 'HCLTECH.NS', 'M&M.NS', 'HDFCBANK.NS', 'TECHM.NS',
       'INDIANB.NS', 'INFY.NS', 'KOTAKBANK.NS', 'HINDUNILVR.NS',
       'DABUR.NS', 'BHARTIARTL.NS', 'JUBLFOOD.NS', 'TATAPOWER.NS']

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
9,JUBLFOOD.NS,487.0,-13.83,-0.18,-1.35,687.0,463.0,10.65,89.35,36.0,636.0,87.9,67.90,31.0,41.0,36.0,1.0,1.0,B
16,BAJFINANCE.NS,5886.0,-12.65,0.47,-1.55,7720.0,5265.0,25.31,74.69,39.0,7391.0,50.8,33.10,26.0,31.0,28.0,1.0,1.0,T
14,INDUSINDBK.NS,1083.0,-2.70,2.16,-0.08,1264.0,771.0,63.28,36.72,28.0,1402.0,22.8,12.40,29.0,17.0,23.0,1.0,1.0,O
11,ICICIBANK.NS,832.0,-3.94,1.27,-0.78,953.0,650.0,60.03,39.97,34.0,1094.0,24.2,18.20,31.0,15.0,23.0,1.0,1.0,O
48,TATAPOWER.NS,213.0,-4.10,0.00,-0.94,287.0,195.0,19.46,80.54,58.0,232.0,20.4,26.70,9.0,35.0,22.0,1.0,0.0,O
25,EICHERMOT.NS,3263.0,-1.88,0.99,-0.70,3850.0,2241.0,63.54,36.46,55.0,3916.0,40.1,38.40,20.0,18.0,19.0,1.0,1.0,T
35,INFY.NS,1534.0,1.91,0.60,-0.32,1866.0,1351.0,35.49,64.51,52.0,1742.0,23.0,27.00,14.0,22.0,18.0,1.0,1.0,A
21,AXISBANK.NS,872.0,6.68,2.20,0.22,962.0,625.0,73.18,26.82,31.0,1060.0,31.3,13.00,22.0,10.0,16.0,1.0,0.0,A
36,AUBANK.NS,619.0,-0.82,0.16,0.00,717.0,532.0,46.97,53.03,45.0,698.0,NaN,121.00,13.0,16.0,14.0,1.0,0.0,D
34,BHARTIARTL.NS,770.0,0.30,0.91,-0.86,851.0,638.0,62.11,37.89,41.0,879.0,NaN,71.50,14.0,11.0,12.0,1.0,1.0,D


In [50]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] == -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
7,IBREALEST.NS,72.0,-6.17,1.30,-1.25,146.0,60.0,14.24,85.76,40.0,95.0,11.3,1000.00,32.0,103.0,68.0,-1.0,-1.0,B
1,INDUSTOWER.NS,153.0,-22.18,-1.52,-2.13,246.0,137.0,14.95,85.05,32.0,213.0,NaN,15.20,39.0,61.0,50.0,-1.0,1.0,O
0,GODREJIND.NS,430.0,-3.46,0.00,0.45,618.0,405.0,11.55,88.45,43.0,638.0,39.5,18.70,48.0,44.0,46.0,0.0,1.0,B
13,NAM-INDIA.NS,249.0,-8.14,-0.74,-0.39,340.0,235.0,13.29,86.71,49.0,322.0,29.9,22.30,29.0,37.0,33.0,-1.0,1.0,O
4,ZEEL.NS,227.0,-8.55,0.00,-1.23,299.0,206.0,22.37,77.63,44.0,303.0,24.6,26.90,33.0,32.0,32.0,-1.0,1.0,O
51,WIPRO.NS,399.0,-1.03,-0.25,0.25,605.0,374.0,10.76,89.24,52.0,425.0,19.0,19.30,7.0,52.0,30.0,-1.0,0.0,O
50,NATIONALUM.NS,83.0,11.13,1.33,1.27,126.0,66.0,28.92,71.08,59.0,90.0,10.3,5.98,8.0,52.0,30.0,0.0,1.0,O
2,PFIZER.NS,3869.0,-9.39,0.02,-1.13,4599.0,3869.0,0.02,99.98,18.0,5297.0,36.5,115.00,37.0,19.0,28.0,0.0,1.0,O
20,PVR.NS,1698.0,-6.88,-0.33,-0.40,2193.0,1498.0,28.72,71.28,49.0,2106.0,51.2,1000.00,24.0,29.0,26.0,1.0,-1.0,O
46,HINDALCO.NS,468.0,10.73,2.36,0.86,626.0,314.0,49.49,50.51,46.0,514.0,10.4,7.60,10.0,34.0,22.0,-1.0,1.0,A
